# SQL CARTO QUERY FUNCTION
# written for PUI2016_Python2

##### This notebook is set up to link as a default to the fb55 account.  To turn in the homework use the same account you used in the lab (hvt201)  and query the database that you were querying in class (citibike_feb_2015)

In [1]:
SQL_SOURCE = 'https://sgo230.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return response.read()

SyntaxError: invalid syntax (<ipython-input-1-b59737a74945>, line 23)

In [ ]:
test = '''
SELECT
    *
FROM
    citibike
'''

In [ ]:
pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',').head()

<b>Task 1</b>

In [ ]:
#Sort data by start_station_id, tripduration
#Only checking trips with duration <= 3 hours 
task1_1 = '''
SELECT
    *
FROM
    citibike
WHERE
    tripduration / 3600 <= 3
ORDER BY
    start_station_id,
    tripduration
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_1)), sep=',').head()

In [ ]:
# Only show the top/last 10 records (aka head and tail in SQL) 
task1_2a = '''
SELECT
    *
FROM
    citibike
WHERE
    tripduration / 3600 <= 3
ORDER BY
    start_station_id, 
    tripduration
LIMIT 10
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2a)), sep=',').head()

In [ ]:
# Only show the top/last 10 records (aka head and tail in SQL) 
task1_2b = '''
SELECT 
    *
FROM 
    citibike
WHERE 
    tripduration / 3600 <= 3
ORDER BY
    start_station_id DESC, 
    tripduration DESC
LIMIT 10
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_2b)), sep=',').head()

In [ ]:
# List all unique start_station_id values
task1_3 = '''
SELECT DISTINCT
    start_station_id
FROM 
    citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_3)), sep=',').head()

In [ ]:
# Count the number of trips (aka wc -l in SQL) 
task1_4 = '''
SELECT
    count(*)
FROM 
    citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_4)), sep=',').head()

In [ ]:
# Find the average/min/max trip duration
task1_5 = '''
SELECT
    avg(tripduration) as Avg_Duration,
    min(tripduration) as Min_Duration,
    max(tripduration) as Max_Duration
FROM 
    citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task1_5)), sep=',').head()

<b>Task 2</b>

In [ ]:
# Selecting trips started on Feb-02-2015 only 
task2_1 = '''
SELECT
    *
FROM 
    citibike
WHERE 
    starttime >= '2015-02-02'
    AND starttime < '2015-02-03'
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_1)), sep=',').head()

In [ ]:
# Selecting trips started on the weekend 
task2_2a = '''
SELECT
    *
FROM 
    citibike
WHERE 
    extract(dow from starttime) in (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2a)), sep=',').head()

In [ ]:
# Selecting trips started on the weekend 
task2_2b = '''
SELECT
    avg(tripduration) as Avg_Duration
FROM 
    citibike
WHERE 
    extract(dow from starttime) in (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_2b)), sep=',').head()

In [ ]:
# Can we do the same for weekday?
task2_3a = '''
SELECT
    *
FROM 
    citibike
WHERE 
    extract(dow from starttime) not in (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3a)), sep=',').head()

In [ ]:
# Can we do the same for weekday?
task2_3b = '''
SELECT
    avg(tripduration) as Avg_Duration_Wd
FROM 
    citibike
WHERE 
    extract(dow from starttime) not in (0,6)
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task2_3b)), sep=',').head()

<b>Task 3</b>

In [ ]:
# Showing the list of start station locations using Group By
task3_1 = '''
SELECT DISTINCT
    concat(start_station_latitude::text,', ',start_station_longitude::text) as start_station_location
FROM 
    citibike
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_1)), sep=',').head()

In [ ]:
# Showing the number of trips started per station 
# but only for stations within 500m of Time Square
# The coordinates of Time Square is (40.7577,-73.9857)
task3_2 = '''
SELECT DISTINCT
    start_station_id,
    start_station_name,
    count(*) as rides
FROM 
    citibike
WHERE
    ST_DWithin(CDB_LatLng(start_station_latitude,start_station_longitude)::geography,CDB_LatLng(40.7577,-73.9857)::geography,500)
GROUP BY 
    start_station_id, start_station_name
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task3_2)), sep=',').head()

<b>Task 4</b>

In [ ]:
#Find the station that had the longest average trip duration during
#weekends and within 500m of TimeSquare! 
task4_1 = '''
SELECT DISTINCT
    start_station_id,
    start_station_name,
    avg(tripduration) as avg_duration
FROM 
    citibike
WHERE
    ST_DWithin(CDB_LatLng(start_station_latitude,start_station_longitude)::geography,CDB_LatLng(40.7577,-73.9857)::geography,500)
    AND extract(dow from starttime) in (0,6)
GROUP BY 
    start_station_id, 
    start_station_name
ORDER BY 
    avg(tripduration) DESC
LIMIT 1
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task4_1)), sep=',').head()

In [ ]:
#Extra: create lines for trips started from stations within 500m of Times
#Squares and lasted less than 2 hours. The number of trips per each
#pair of stations are output as attributes of these lines. 
task4_2 = '''
SELECT DISTINCT
    start_station_id,
    start_station_name,
    avg(tripduration) as avg_duration,
    min(cartodb_id) as cartodb_id,
    ST_Makeline(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude,start_station_longitude)),
    CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude,end_station_longitude))) as the_geom_webmercator
FROM 
    citibike
WHERE
    ST_DWithin(CDB_LatLng(start_station_latitude,start_station_longitude)::geography,CDB_LatLng(40.7577,-73.9857)::geography,500)
    AND tripduration / 3600 < 2
GROUP BY 
    start_station_id, 
    start_station_name,
    start_station_latitude,
    start_station_longitude,
    end_station_latitude,
    end_station_longitude
'''
pd.read_csv(StringIO.StringIO(queryCartoDB(task4_2)), sep=',').head()

Map link for above:
https://sgo230.carto.com/builder/18ee1175-2808-4278-a9e4-4b89b830b08a/embed
<br><br>
Screenshot:
![title](carto_screenshot.PNG)

## ASH - Score 100. Extra Credit 0